# PySpark String Matching
### In this exercise we have receipts from 2 different sources. The task is to find receipts that are the same and flag them to be removed in the future.

### Flagging by receipt total and store is not enough, therefore item description needs to be used. Due to different OCR approaches, item descriptions of the same receipts may not be identical (example: 'banana' vs 'bananas'). This needs to be accounted for.

# Libraires and paths

In [1]:
#Importing libraries
import findspark
findspark.init()
import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-21"

from fuzzywuzzy import fuzz

#Initializing Spark session
spark = SparkSession.builder.master('local[1]').appName('string_matching').getOrCreate()

D:\Program_Files\Anaconda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Reading in the data

In [2]:
#Reading in the data
df = spark.read.csv('dummy_receipt_file.csv', header='true')

In [3]:
#Checking top rows
df.show()

+-------+------------+-------+-------------+------+----------------+
| source|  receipt_id|user_id|receipt_total| store|item_description|
+-------+------------+-------+-------------+------+----------------+
|source1|s1u110store1|   s1u1|           10|store1|           apple|
|source1|s1u110store1|   s1u1|           10|store1|          banana|
|source1|s1u217store2|   s1u2|           17|store2|             tea|
|source1|s1u217store2|   s1u2|           17|store2|           sugar|
|source1|s1u217store2|   s1u2|           17|store2|       paper bag|
|source1|s1u325store3|   s1u3|           25|store3|        water 1L|
|source1|s1u325store3|   s1u3|           25|store3|    orange juice|
|source1|s1u325store3|   s1u3|           25|store3|     plastic bag|
|source2|s2u110store1|   s2u1|           10|store1|         bananas|
|source2|s2u110store1|   s2u1|           10|store1|          apples|
|source2|s2u217store2|   s2u2|           17|store2|             tea|
|source2|s2u217store2|   s2u2|    

In [4]:
#Splitting the data into 2 sources
df_source_1 = df.filter(col('source') == 'source1')
df_source_2 = df.filter(col('source') == 'source2')

In [5]:
df_source_1.show()

+-------+------------+-------+-------------+------+----------------+
| source|  receipt_id|user_id|receipt_total| store|item_description|
+-------+------------+-------+-------------+------+----------------+
|source1|s1u110store1|   s1u1|           10|store1|           apple|
|source1|s1u110store1|   s1u1|           10|store1|          banana|
|source1|s1u217store2|   s1u2|           17|store2|             tea|
|source1|s1u217store2|   s1u2|           17|store2|           sugar|
|source1|s1u217store2|   s1u2|           17|store2|       paper bag|
|source1|s1u325store3|   s1u3|           25|store3|        water 1L|
|source1|s1u325store3|   s1u3|           25|store3|    orange juice|
|source1|s1u325store3|   s1u3|           25|store3|     plastic bag|
+-------+------------+-------+-------------+------+----------------+



In [6]:
df_source_2.show()

+-------+------------+-------+-------------+------+----------------+
| source|  receipt_id|user_id|receipt_total| store|item_description|
+-------+------------+-------+-------------+------+----------------+
|source2|s2u110store1|   s2u1|           10|store1|         bananas|
|source2|s2u110store1|   s2u1|           10|store1|          apples|
|source2|s2u217store2|   s2u2|           17|store2|             tea|
|source2|s2u217store2|   s2u2|           17|store2|           sugar|
|source2|s2u217store2|   s2u2|           17|store2|       paper bag|
|source2|s2u370store4|   s2u3|           70|store4|       ice cream|
|source2|s2u370store4|   s2u3|           70|store4|        potatoes|
|source2|s2u370store4|   s2u3|           70|store4|        water 1L|
|source2|s2u370store4|   s2u3|           70|store4|        water 1L|
|source2|s2u370store4|   s2u3|           70|store4|        water 1L|
|source2|s2u370store4|   s2u3|           70|store4|        water 1L|
+-------+------------+-------+----

# Transforming the data

In [7]:
#Groupping the receipts by ID and generating a list of items
df_source_1 = df_source_1.groupBy('receipt_id', 'user_id', 'receipt_total', 'store').agg(collect_list('item_description').alias('s1_item_desc')).withColumnRenamed('receipt_id','s1_receipt_id').withColumnRenamed('user_id','s1_user_id')
df_source_2 = df_source_2.groupBy('receipt_id', 'user_id', 'receipt_total', 'store').agg(collect_list('item_description').alias('s2_item_desc')).withColumnRenamed('receipt_id','s2_receipt_id').withColumnRenamed('user_id','s2_user_id')

In [8]:
df_source_1.show()

+-------------+----------+-------------+------+--------------------+
|s1_receipt_id|s1_user_id|receipt_total| store|        s1_item_desc|
+-------------+----------+-------------+------+--------------------+
| s1u217store2|      s1u2|           17|store2|[tea, sugar, pape...|
| s1u325store3|      s1u3|           25|store3|[water 1L, orange...|
| s1u110store1|      s1u1|           10|store1|     [apple, banana]|
+-------------+----------+-------------+------+--------------------+



In [9]:
df_source_2.show()

+-------------+----------+-------------+------+--------------------+
|s2_receipt_id|s2_user_id|receipt_total| store|        s2_item_desc|
+-------------+----------+-------------+------+--------------------+
| s2u370store4|      s2u3|           70|store4|[ice cream, potat...|
| s2u217store2|      s2u2|           17|store2|[tea, sugar, pape...|
| s2u110store1|      s2u1|           10|store1|   [bananas, apples]|
+-------------+----------+-------------+------+--------------------+



# Matching receipts from the same stores with the same totals

In [10]:
df_matched_receipts = df_source_1.join(df_source_2, how='inner', on=['receipt_total', 'store'])
df_matched_receipts.show()

+-------------+------+-------------+----------+--------------------+-------------+----------+--------------------+
|receipt_total| store|s1_receipt_id|s1_user_id|        s1_item_desc|s2_receipt_id|s2_user_id|        s2_item_desc|
+-------------+------+-------------+----------+--------------------+-------------+----------+--------------------+
|           17|store2| s1u217store2|      s1u2|[tea, sugar, pape...| s2u217store2|      s2u2|[tea, sugar, pape...|
|           10|store1| s1u110store1|      s1u1|     [apple, banana]| s2u110store1|      s2u1|   [bananas, apples]|
+-------------+------+-------------+----------+--------------------+-------------+----------+--------------------+



# Run the string matching for receipts that are potentially duplicates

In [11]:
#Fuzzy match UDF
def matchstring(s1, s2):
    return fuzz.token_sort_ratio(s1, s2)
MatchUDF = udf(matchstring, StringType())

df_matched_receipts = df_matched_receipts.withColumn("similarity_score", MatchUDF(col("s1_item_desc"), col("s2_item_desc")))


In [16]:
df_matched_receipts.select('s1_receipt_id','s2_receipt_id', 's1_item_desc', 's2_item_desc', 'similarity_score').show()

+-------------+-------------+--------------------+--------------------+----------------+
|s1_receipt_id|s2_receipt_id|        s1_item_desc|        s2_item_desc|similarity_score|
+-------------+-------------+--------------------+--------------------+----------------+
| s1u217store2| s2u217store2|[tea, sugar, pape...|[tea, sugar, pape...|             100|
| s1u110store1| s2u110store1|     [apple, banana]|   [bananas, apples]|              92|
+-------------+-------------+--------------------+--------------------+----------------+



In [17]:
#APPLYING FUZZY SCORE LABEL - considering score of 80 and above to me a match
df_matched_receipts = df_matched_receipts.withColumn('if_same_receipts', when((col('similarity_score') >= 80), "same").otherwise("not_same"))
df_matched_receipts.select('s1_receipt_id','s2_receipt_id', 's1_item_desc', 's2_item_desc', 'similarity_score', 'if_same_receipts').show()

+-------------+-------------+--------------------+--------------------+----------------+----------------+
|s1_receipt_id|s2_receipt_id|        s1_item_desc|        s2_item_desc|similarity_score|if_same_receipts|
+-------------+-------------+--------------------+--------------------+----------------+----------------+
| s1u217store2| s2u217store2|[tea, sugar, pape...|[tea, sugar, pape...|             100|            same|
| s1u110store1| s2u110store1|     [apple, banana]|   [bananas, apples]|              92|            same|
+-------------+-------------+--------------------+--------------------+----------------+----------------+



# Done! Receipts with similarity score of 80 and above are confirmed to be the same receipt from 2 different sources.